In [4]:
import requests
import pandas as pd
import time
import re
from datetime import datetime, timedelta

# ============================================================
# CONFIGURATION
# ============================================================

SEC_SEARCH_URL = "https://efts.sec.gov/LATEST/search-index"

HEADERS = {
    "User-Agent": "B. Dyson Capital Advisors contact@bdysoncapital.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "efts.sec.gov"
}

# Comprehensive filing types
COMPREHENSIVE_FILINGS = [
    # Core capital raising
    "S-1", "S-3", "S-4", "S-8",
    "S-1/A", "S-3/A", "S-4/A", "S-8/A",
    "S-3ASR", "S-1MEF", "S-4MEF",
    "POS AM", "POSASR",

    # Prospectuses
    "424B1", "424B3", "424B5",

    # Private placements
    "D", "D/A",

    # M&A work
    "SC TO-I", "SC TO-I/A",
    "SC 13E3", "SC 13E4",

    # Governance/proxy
    "DEF 14A", "DEFA14A",
    "DEFM14A",

    # Material events
    "8-K", "8-K/A",

    # Financial reports
    "10-K", "10-Q", "10-K/A", "10-Q/A",

    # Investor filings
    "SC 13D", "SC 13G", "SC 13D/A", "SC 13G/A",

    # Registration effectiveness
    "EFFECT",
]

TARGET_COMPANIES = 100

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def extract_ticker_and_clean_name(company_name):
    """Extract ticker from company name"""
    name_without_cik = re.sub(r'\s*\(CIK\s+\d+\)', '', company_name)
    ticker_match = re.search(r'\(([A-Z0-9\-]+)', name_without_cik)
    ticker = ticker_match.group(1) if ticker_match else ""
    clean_name = re.split(r'\s*\(', name_without_cik)[0].strip()
    return clean_name, ticker

# ============================================================
# SEARCH FUNCTIONS
# ============================================================

def search_edgar(search_term, from_date, to_date, start_index=0, max_results=100):
    """Search SEC EDGAR"""
    results = []
    from_str = from_date.strftime("%Y-%m-%d")
    to_str = to_date.strftime("%Y-%m-%d")

    params = {
        "q": f'"{search_term}"',
        "dateRange": "custom",
        "startdt": from_str,
        "enddt": to_str,
        "from": start_index,
        "size": min(max_results, 100)
    }

    try:
        response = requests.get(SEC_SEARCH_URL, params=params, headers=HEADERS)
        response.raise_for_status()
        data = response.json()

        if "hits" in data and "hits" in data["hits"]:
            total_hits = data["hits"]["total"]["value"]

            for hit in data["hits"]["hits"]:
                source = hit.get("_source", {})
                filing_info = {
                    "search_term": search_term,
                    "company_name": source.get("display_names", ["Unknown"])[0] if source.get("display_names") else "Unknown",
                    "filing_type": source.get("file_type", ""),
                    "filing_date": source.get("file_date", ""),
                }
                results.append(filing_info)

            return results, total_hits

    except requests.exceptions.RequestException as e:
        print(f"Error searching: {e}")

    time.sleep(0.15)
    return results, 0

def search_paginated(search_term, from_date, to_date, max_total=500):
    """Search with pagination"""
    all_results = []
    start_index = 0
    page_size = 100

    while len(all_results) < max_total:
        results, total = search_edgar(search_term, from_date, to_date, start_index, page_size)
        if not results:
            break
        all_results.extend(results)
        start_index += page_size
        if len(all_results) >= total:
            break

    return all_results, total

def count_unique_companies(search_term, from_date, to_date):
    """Count unique companies in a date range"""
    results, _ = search_paginated(search_term, from_date, to_date, max_total=500)

    if not results:
        return 0

    df = pd.DataFrame(results)

    # Filter to comprehensive filing types
    df_filtered = df[df["filing_type"].isin(COMPREHENSIVE_FILINGS)].copy()

    # Extract company names
    df_filtered[['clean_company_name', 'ticker']] = df_filtered['company_name'].apply(
        lambda x: pd.Series(extract_ticker_and_clean_name(x))
    )

    # Count unique companies with tickers
    unique_with_tickers = df_filtered[df_filtered['ticker'] != '']['clean_company_name'].nunique()

    return unique_with_tickers

def determine_optimal_date_range(search_term):
    """
    Adaptive date range to keep results under 100 companies
    Options: 2 years, 3-5 years, 6-8 years
    """
    end_date = datetime.now()

    print(f"\nTesting volume for '{search_term}'...")

    # Test 2 years first
    test_2yr = end_date - timedelta(days=730)
    count_2yr = count_unique_companies(search_term, test_2yr, end_date)
    print(f"  2 years: {count_2yr} unique companies")

    # Decide based on 2-year count
    if count_2yr >= TARGET_COMPANIES:
        # High volume - use 2 years
        days = 730
        final_range = "2 years"
        print(f"HIGH VOLUME - Using 2 years")
    elif count_2yr >= 40:
        # Good range - use 2 years
        days = 730
        final_range = "2 years"
        print(f"OPTIMAL VOLUME - Using 2 years")
    else:
        # Test 4 years
        test_4yr = end_date - timedelta(days=1460)
        count_4yr = count_unique_companies(search_term, test_4yr, end_date)
        print(f"  4 years: {count_4yr} unique companies")

        if count_4yr >= TARGET_COMPANIES:
            # Use 3 years
            days = 1095
            final_range = "3 years"
            print(f"MEDIUM-HIGH VOLUME - Using 3 years")
        elif count_4yr >= 30:
            # Use 4 years
            days = 1460
            final_range = "4 years"
            print(f"MEDIUM VOLUME - Using 4 years")
        elif count_4yr >= 15:
            # Use 5 years
            days = 1825
            final_range = "5 years"
            print(f"MEDIUM VOLUME - Using 5 years")
        else:
            # Low volume - use 7 years
            days = 2555
            final_range = "7 years"
            print(f"LOW VOLUME - Using 7 years for better coverage")

    start_date = end_date - timedelta(days=days)
    return start_date, end_date, final_range

def filter_important_filings(df):
    """Filter for comprehensive filing types"""
    if df.empty:
        return df
    return df[df["filing_type"].isin(COMPREHENSIVE_FILINGS)].copy()

def deduplicate_companies(df):
    """Keep only most recent filing per company"""
    if df.empty:
        return df
    df = df.sort_values('filing_date', ascending=False)
    df_unique = df.drop_duplicates(subset=['clean_company_name'], keep='first')
    return df_unique

# ============================================================
# MAIN SEARCH FUNCTION
# ============================================================

def search_lawyer(lawyer_name):
    """
    Search for companies represented by a lawyer
    Adaptively adjusts date range to keep under 100 companies
    """

    print("=" * 70)
    print(f"SEARCHING LAWYER: {lawyer_name}")
    print("=" * 70)

    # Determine optimal date range
    start_date, end_date, date_range_desc = determine_optimal_date_range(lawyer_name)

    print(f"\nFinal search: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} ({date_range_desc})")

    # Search with determined date range
    results, total = search_paginated(lawyer_name, start_date, end_date, max_total=500)

    if not results:
        print("No results found.")
        return pd.DataFrame()

    df = pd.DataFrame(results)
    print(f"\nTotal filings found: {len(df)}")

    # Filter to comprehensive filing types
    df_filtered = filter_important_filings(df)
    print(f"After filtering to relevant filing types: {len(df_filtered)}")

    if df_filtered.empty:
        print("No relevant filings after filtering.")
        return pd.DataFrame()

    # Extract ticker and clean name
    df_filtered[['clean_company_name', 'ticker']] = df_filtered['company_name'].apply(
        lambda x: pd.Series(extract_ticker_and_clean_name(x))
    )

    # Convert dates
    df_filtered['filing_date'] = pd.to_datetime(df_filtered['filing_date'])

    # Deduplicate
    df_unique = deduplicate_companies(df_filtered)
    print(f"Unique companies: {len(df_unique)}")

    # Sort by most recent
    df_unique = df_unique.sort_values('filing_date', ascending=False)

    # Keep only relevant columns
    result_df = df_unique[['clean_company_name', 'ticker']].copy()
    result_df.columns = ['Company', 'Ticker']

    # Remove companies without tickers
    result_df = result_df[result_df['Ticker'] != ""].copy()

    print(f"\n✓ Search complete: {len(result_df)} companies with tickers")

    return result_df

# ============================================================
# EXPORT FUNCTION
# ============================================================

# TICKERS ONLY
# def export_tickers_csv(df, filename):
#     """Export single-column CSV with Bloomberg tickers"""
#     if df.empty:
#         print("\nDataFrame is empty, nothing to export.")
#         return []

#     # Get unique tickers
#     tickers = df['Ticker'].dropna().unique()
#     tickers = [t for t in tickers if t != "" and t != "Private"]

#     if len(tickers) == 0:
#         print("\nNo valid tickers to export.")
#         return []

#     # Create Bloomberg format
#     bloomberg_tickers = [f"{ticker} US Equity" for ticker in tickers]

#     # Create single-column DataFrame
#     export_df = pd.DataFrame({'Ticker': bloomberg_tickers})

#     # Export to CSV
#     export_df.to_csv(filename, index=False)

#     print(f"\n✓ Exported to: {filename}")
#     print(f"✓ Total tickers: {len(bloomberg_tickers)}")
#     print(f"\nFirst 5 tickers:")
#     for ticker in bloomberg_tickers[:5]:
#         print(f"  {ticker}")

#     return bloomberg_tickers

# NAMES AND TICKERS
def export_tickers_csv(df, filename):
    """Export CSV with company names, Bloomberg tickers, and most recent filing date"""
    if df.empty:
        print("\nDataFrame is empty, nothing to export.")
        return []

    # Remove rows without valid tickers
    df_valid = df[
        (df['Ticker'].notna()) &
        (df['Ticker'] != "") &
        (df['Ticker'] != "Private")
    ].copy()

    if len(df_valid) == 0:
        print("\nNo valid tickers to export.")
        return []

    # Create Bloomberg format ticker column
    df_valid['Bloomberg_Ticker'] = df_valid['Ticker'].apply(lambda x: f"{x} US Equity")

    # Create export DataFrame with Company, Bloomberg Ticker, and Filing Date
    export_df = df_valid[['Company', 'Bloomberg_Ticker', 'filing_date']].copy()
    export_df.columns = ['Company', 'Ticker', 'Date of most recent filing for this company']

    # Format the date column (convert to just date, no time)
    export_df['Date of most recent filing for this company'] = pd.to_datetime(
        export_df['Date of most recent filing for this company']
    ).dt.strftime('%Y-%m-%d')

    # Sort alphabetically by ticker
    export_df = export_df.sort_values('Ticker')

    # Export to CSV
    export_df.to_csv(filename, index=False)

    print(f"\n✓ Exported to: {filename}")
    print(f"✓ Total companies: {len(export_df)}")
    print(f"\nFirst 5 entries:")
    print(export_df.head().to_string(index=False))

    return export_df['Ticker'].tolist()

In [8]:
import requests
import pandas as pd
import time
import re
from datetime import datetime, timedelta

# ============================================================
# CONFIGURATION
# ============================================================

SEC_SEARCH_URL = "https://efts.sec.gov/LATEST/search-index"

HEADERS = {
    "User-Agent": "B. Dyson Capital Advisors contact@bdysoncapital.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "efts.sec.gov"
}

# Comprehensive filing types
COMPREHENSIVE_FILINGS = [
    # Core capital raising
    "S-1", "S-3", "S-4", "S-8",
    "S-1/A", "S-3/A", "S-4/A", "S-8/A",
    "S-3ASR", "S-1MEF", "S-4MEF",
    "POS AM", "POSASR",

    # Prospectuses
    "424B1", "424B3", "424B5",

    # Private placements
    "D", "D/A",

    # M&A work
    "SC TO-I", "SC TO-I/A",
    "SC 13E3", "SC 13E4",

    # Governance/proxy
    "DEF 14A", "DEFA14A",
    "DEFM14A",

    # Material events
    "8-K", "8-K/A",

    # Financial reports
    "10-K", "10-Q", "10-K/A", "10-Q/A",

    # Investor filings
    "SC 13D", "SC 13G", "SC 13D/A", "SC 13G/A",

    # Registration effectiveness
    "EFFECT",
]

TARGET_COMPANIES = 100

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def extract_ticker_and_clean_name(company_name):
    """Extract ticker from company name"""
    name_without_cik = re.sub(r'\s*\(CIK\s+\d+\)', '', company_name)
    ticker_match = re.search(r'\(([A-Z0-9\-]+)', name_without_cik)
    ticker = ticker_match.group(1) if ticker_match else ""
    clean_name = re.split(r'\s*\(', name_without_cik)[0].strip()
    return clean_name, ticker

# ============================================================
# SEARCH FUNCTIONS
# ============================================================

def search_edgar(search_term, from_date, to_date, start_index=0, max_results=100):
    """Search SEC EDGAR"""
    results = []
    from_str = from_date.strftime("%Y-%m-%d")
    to_str = to_date.strftime("%Y-%m-%d")

    params = {
        "q": f'"{search_term}"',
        "dateRange": "custom",
        "startdt": from_str,
        "enddt": to_str,
        "from": start_index,
        "size": min(max_results, 100)
    }

    try:
        response = requests.get(SEC_SEARCH_URL, params=params, headers=HEADERS)
        response.raise_for_status()
        data = response.json()

        if "hits" in data and "hits" in data["hits"]:
            total_hits = data["hits"]["total"]["value"]

            for hit in data["hits"]["hits"]:
                source = hit.get("_source", {})
                filing_info = {
                    "search_term": search_term,
                    "company_name": source.get("display_names", ["Unknown"])[0] if source.get("display_names") else "Unknown",
                    "filing_type": source.get("file_type", ""),
                    "filing_date": source.get("file_date", ""),
                }
                results.append(filing_info)

            return results, total_hits

    except requests.exceptions.RequestException as e:
        print(f"Error searching: {e}")

    time.sleep(0.15)
    return results, 0

def search_paginated(search_term, from_date, to_date, max_total=500):
    """Search with pagination"""
    all_results = []
    start_index = 0
    page_size = 100

    while len(all_results) < max_total:
        results, total = search_edgar(search_term, from_date, to_date, start_index, page_size)
        if not results:
            break
        all_results.extend(results)
        start_index += page_size
        if len(all_results) >= total:
            break

    return all_results, total

def count_unique_companies(search_term, from_date, to_date):
    """Count unique companies in a date range"""
    results, _ = search_paginated(search_term, from_date, to_date, max_total=500)

    if not results:
        return 0

    df = pd.DataFrame(results)

    # Filter to comprehensive filing types
    df_filtered = df[df["filing_type"].isin(COMPREHENSIVE_FILINGS)].copy()

    # Extract company names
    df_filtered[['clean_company_name', 'ticker']] = df_filtered['company_name'].apply(
        lambda x: pd.Series(extract_ticker_and_clean_name(x))
    )

    # Count unique companies with tickers
    unique_with_tickers = df_filtered[df_filtered['ticker'] != '']['clean_company_name'].nunique()

    return unique_with_tickers

def determine_optimal_date_range(search_term):
    """
    Adaptive date range to keep results under 100 companies
    Options: 2 years, 3-5 years, 6-8 years
    """
    end_date = datetime.now()

    print(f"\nTesting volume for '{search_term}'...")

    # Test 2 years first
    test_2yr = end_date - timedelta(days=730)
    count_2yr = count_unique_companies(search_term, test_2yr, end_date)
    print(f"  2 years: {count_2yr} unique companies")

    # Decide based on 2-year count
    if count_2yr >= TARGET_COMPANIES:
        # High volume - use 2 years
        days = 730
        final_range = "2 years"
        print(f"HIGH VOLUME - Using 2 years")
    elif count_2yr >= 40:
        # Good range - use 2 years
        days = 730
        final_range = "2 years"
        print(f"OPTIMAL VOLUME - Using 2 years")
    else:
        # Test 4 years
        test_4yr = end_date - timedelta(days=1460)
        count_4yr = count_unique_companies(search_term, test_4yr, end_date)
        print(f"  4 years: {count_4yr} unique companies")

        if count_4yr >= TARGET_COMPANIES:
            # Use 3 years
            days = 1095
            final_range = "3 years"
            print(f"MEDIUM-HIGH VOLUME - Using 3 years")
        elif count_4yr >= 30:
            # Use 4 years
            days = 1460
            final_range = "4 years"
            print(f"MEDIUM VOLUME - Using 4 years")
        elif count_4yr >= 15:
            # Use 5 years
            days = 1825
            final_range = "5 years"
            print(f"MEDIUM VOLUME - Using 5 years")
        else:
            # Low volume - use 7 years
            days = 2555
            final_range = "7 years"
            print(f"LOW VOLUME - Using 7 years for better coverage")

    start_date = end_date - timedelta(days=days)
    return start_date, end_date, final_range

def filter_important_filings(df):
    """Filter for comprehensive filing types"""
    if df.empty:
        return df
    return df[df["filing_type"].isin(COMPREHENSIVE_FILINGS)].copy()

def deduplicate_companies(df):
    """Keep only most recent filing per company"""
    if df.empty:
        return df
    df = df.sort_values('filing_date', ascending=False)
    df_unique = df.drop_duplicates(subset=['clean_company_name'], keep='first')
    return df_unique

# ============================================================
# MAIN SEARCH FUNCTION
# ============================================================

def search_lawyer(lawyer_name):
    """
    Search for companies represented by a lawyer
    Adaptively adjusts date range to keep under 100 companies
    """

    print("=" * 70)
    print(f"SEARCHING LAWYER: {lawyer_name}")
    print("=" * 70)

    # Determine optimal date range
    start_date, end_date, date_range_desc = determine_optimal_date_range(lawyer_name)

    print(f"\nFinal search: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} ({date_range_desc})")

    # Search with determined date range
    results, total = search_paginated(lawyer_name, start_date, end_date, max_total=500)

    if not results:
        print("No results found.")
        return pd.DataFrame()

    df = pd.DataFrame(results)
    print(f"\nTotal filings found: {len(df)}")

    # Filter to comprehensive filing types
    df_filtered = filter_important_filings(df)
    print(f"After filtering to relevant filing types: {len(df_filtered)}")

    if df_filtered.empty:
        print("No relevant filings after filtering.")
        return pd.DataFrame()

    # Extract ticker and clean name
    df_filtered[['clean_company_name', 'ticker']] = df_filtered['company_name'].apply(
        lambda x: pd.Series(extract_ticker_and_clean_name(x))
    )

    # Convert dates
    df_filtered['filing_date'] = pd.to_datetime(df_filtered['filing_date'])

    # Deduplicate
    df_unique = deduplicate_companies(df_filtered)
    print(f"Unique companies: {len(df_unique)}")

    # Sort by most recent
    df_unique = df_unique.sort_values('filing_date', ascending=False)

    # Keep only relevant columns (include filing_date)
    result_df = df_unique[['clean_company_name', 'ticker', 'filing_date']].copy()
    result_df.columns = ['Company', 'Ticker', 'filing_date']

    # Remove companies without tickers
    result_df = result_df[result_df['Ticker'] != ""].copy()

    print(f"\n✓ Search complete: {len(result_df)} companies with tickers")

    return result_df

# ============================================================
# EXPORT FUNCTION
# ============================================================

def export_tickers_csv(df, filename):
    """Export CSV with company names, Bloomberg tickers, and most recent filing date"""
    if df.empty:
        print("\nDataFrame is empty, nothing to export.")
        return []

    # Remove rows without valid tickers
    df_valid = df[
        (df['Ticker'].notna()) &
        (df['Ticker'] != "") &
        (df['Ticker'] != "Private")
    ].copy()

    if len(df_valid) == 0:
        print("\nNo valid tickers to export.")
        return []

    # Create Bloomberg format ticker column
    df_valid['Bloomberg_Ticker'] = df_valid['Ticker'].apply(lambda x: f"{x} US Equity")

    # Create export DataFrame with Company, Bloomberg Ticker, and Filing Date
    export_df = df_valid[['Company', 'Bloomberg_Ticker', 'filing_date']].copy()
    export_df.columns = ['Company', 'Ticker', 'Date of most recent filing for this company']

    # Format the date column (convert to just date, no time)
    export_df['Date of most recent filing for this company'] = pd.to_datetime(
        export_df['Date of most recent filing for this company']
    ).dt.strftime('%Y-%m-%d')

    # Sort alphabetically by ticker
    export_df = export_df.sort_values('Ticker')

    # Export to CSV
    export_df.to_csv(filename, index=False)

    print(f"\n✓ Exported to: {filename}")
    print(f"✓ Total companies: {len(export_df)}")
    print(f"\nFirst 5 entries:")
    print(export_df.head().to_string(index=False))

    return export_df['Ticker'].tolist()

In [13]:
# ============================================================
# USAGE
# ============================================================

lawyer_name = "mitchell bloom"
df = search_lawyer(lawyer_name)

if not df.empty:
    filename = f"{lawyer_name.lower().replace(' ', '_')}_tickers.csv"
    tickers = export_tickers_csv(df, filename)

SEARCHING LAWYER: mitchell bloom

Testing volume for 'mitchell bloom'...
  2 years: 2 unique companies
  4 years: 2 unique companies
LOW VOLUME - Using 7 years for better coverage

Final search: 2019-01-22 to 2026-01-20 (7 years)

Total filings found: 92
After filtering to relevant filing types: 30
Unique companies: 7

✓ Search complete: 4 companies with tickers

✓ Exported to: mitchell_bloom_tickers.csv
✓ Total companies: 4

First 5 entries:
                         Company         Ticker Date of most recent filing for this company
        Akero Therapeutics, Inc. AKRO US Equity                                  2020-07-07
Black Diamond Therapeutics, Inc. BDTX US Equity                                  2020-01-29
      Fulcrum Therapeutics, Inc. FULC US Equity                                  2025-02-25
                   AVROBIO, Inc. TECX US Equity                                  2024-04-29
